In [20]:
import scripts.proj1_helpers as helper
import implementations as imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Feature engineering

In [21]:
df = pd.read_csv('train.csv')

In [22]:
dfb = pd.read_csv('test.csv')

In [23]:
len(df[df == -999.0])

250000

In [24]:
for i, c in enumerate(df.columns):
    print(i, c)

0 Id
1 Prediction
2 DER_mass_MMC
3 DER_mass_transverse_met_lep
4 DER_mass_vis
5 DER_pt_h
6 DER_deltaeta_jet_jet
7 DER_mass_jet_jet
8 DER_prodeta_jet_jet
9 DER_deltar_tau_lep
10 DER_pt_tot
11 DER_sum_pt
12 DER_pt_ratio_lep_tau
13 DER_met_phi_centrality
14 DER_lep_eta_centrality
15 PRI_tau_pt
16 PRI_tau_eta
17 PRI_tau_phi
18 PRI_lep_pt
19 PRI_lep_eta
20 PRI_lep_phi
21 PRI_met
22 PRI_met_phi
23 PRI_met_sumet
24 PRI_jet_num
25 PRI_jet_leading_pt
26 PRI_jet_leading_eta
27 PRI_jet_leading_phi
28 PRI_jet_subleading_pt
29 PRI_jet_subleading_eta
30 PRI_jet_subleading_phi
31 PRI_jet_all_pt


In [25]:
df.PRI_jet_num.unique()

array([2, 1, 0, 3])

## Import the data

boson is -1

not boson is 1

In [26]:
y_train, x_train, ids_train = helper.load_csv_data('train.csv')
y_test, x_test, ids_test = helper.load_csv_data('test.csv')

(250000, 30)
(568238, 30)


In [27]:
print('Number of boson:', np.count_nonzero(y_train-1))
print('Number of other:', np.count_nonzero(y_train+1))

Number of boson: 164333
Number of other: 85667


In [32]:
y_train[y_train < 0] = 0

## Logistic regression per category

In [53]:
start = time.perf_counter()
accuracy = imp.logistic_cross_validation(y_train, x_train, k_fold=250)

Current iteration=0, the loss=68977.8485262424
Current iteration=1, the loss=68932.51986161005
Current iteration=2, the loss=68887.28175524631
Current iteration=3, the loss=68797.03126444355
Current iteration=4, the loss=68662.26211527373
Current iteration=5, the loss=68394.7309606622
Current iteration=6, the loss=67911.27372621435
Current iteration=7, the loss=67054.98354041662
Current iteration=8, the loss=65584.37811401437
Current iteration=9, the loss=63140.99437284224
Current iteration=10, the loss=59399.56937664053
Current iteration=11, the loss=54289.181285149316
Current iteration=12, the loss=48465.68274430426
Current iteration=13, the loss=43257.3739884387
Current iteration=14, the loss=39970.08979360185
Current iteration=15, the loss=39017.45295133925
Current iteration=16, the loss=38947.95231161101
Current iteration=17, the loss=38947.396117023476
Current iteration=0, the loss=53524.825282838974
Current iteration=1, the loss=53505.17611223441
Current iteration=2, the loss=53

Current iteration=19, the loss=27567.011646858453
Current iteration=20, the loss=26779.089474076904
Current iteration=21, the loss=26257.286996992203
Current iteration=22, the loss=25987.437482732323
Current iteration=23, the loss=25890.144462286353
Current iteration=24, the loss=25868.71857319682
Current iteration=25, the loss=25866.008007578
Current iteration=26, the loss=25865.785115091694
Current iteration=27, the loss=25865.770139464727
Current iteration=28, the loss=25865.769206454795
Current iteration=0, the loss=15301.917158041353
Current iteration=1, the loss=15295.719077381222
Current iteration=2, the loss=15289.533382076701
Current iteration=3, the loss=15283.360038036193
Current iteration=4, the loss=15271.044148227855
Current iteration=5, the loss=15252.653120796987
Current iteration=6, the loss=15222.21466013501
Current iteration=7, the loss=15174.067635802778
Current iteration=8, the loss=15097.253129658864
Current iteration=9, the loss=14976.795956042039
Current iterati

Current iteration=18, the loss=45503.419264348835
Current iteration=19, the loss=44183.08314743419
Current iteration=20, the loss=43107.37337375026
Current iteration=21, the loss=42370.84680542411
Current iteration=22, the loss=41973.15892854593
Current iteration=23, the loss=41816.16676783526
Current iteration=24, the loss=41774.11802539455
Current iteration=25, the loss=41766.46140543129
Current iteration=26, the loss=41765.411702584934
Current iteration=27, the loss=41765.28722943625
Current iteration=28, the loss=41765.27338155506
Current iteration=29, the loss=41765.271875678445
Current iteration=30, the loss=41765.27171242274
Current iteration=0, the loss=34775.887195873016
Current iteration=1, the loss=34761.370262559016
Current iteration=2, the loss=34746.88233194833
Current iteration=3, the loss=34732.42332048503
Current iteration=4, the loss=34717.993152361094
Current iteration=5, the loss=34689.204731283855
Current iteration=6, the loss=34646.21540697438
Current iteration=7,

Current iteration=4, the loss=68637.43250333061
Current iteration=5, the loss=68370.00618915122
Current iteration=6, the loss=67886.73843113237
Current iteration=7, the loss=67030.78345751428
Current iteration=8, the loss=65560.75237006116
Current iteration=9, the loss=63118.31446947258
Current iteration=10, the loss=59378.288663632666
Current iteration=11, the loss=54269.56410320113
Current iteration=12, the loss=48447.08412475831
Current iteration=13, the loss=43237.78593546616
Current iteration=14, the loss=39945.4179164038
Current iteration=15, the loss=38993.14311132945
Current iteration=16, the loss=38923.694335599794
Current iteration=17, the loss=38923.139409806536
Current iteration=0, the loss=53541.46081517242
Current iteration=1, the loss=53521.80816720066
Current iteration=2, the loss=53502.194755953395
Current iteration=3, the loss=53482.62037678832
Current iteration=4, the loss=53463.085395271155
Current iteration=5, the loss=53424.11176149109
Current iteration=6, the los

Current iteration=23, the loss=25890.259299140398
Current iteration=24, the loss=25869.04194362892
Current iteration=25, the loss=25866.375801655697
Current iteration=26, the loss=25866.15831214019
Current iteration=27, the loss=25866.14382741106
Current iteration=28, the loss=25866.142925171815
Current iteration=0, the loss=15306.076041124712
Current iteration=1, the loss=15299.865587787517
Current iteration=2, the loss=15293.66754537347
Current iteration=3, the loss=15287.481878855857
Current iteration=4, the loss=15275.141403079491
Current iteration=5, the loss=15256.713665962625
Current iteration=6, the loss=15226.214446721768
Current iteration=7, the loss=15177.971285892976
Current iteration=8, the loss=15101.003369600436
Current iteration=9, the loss=14980.305357818383
Current iteration=10, the loss=14793.983221904662
Current iteration=11, the loss=14514.734194105658
Current iteration=12, the loss=14123.199064613267
Current iteration=13, the loss=13614.891381323356
Current iterat

...

Current iteration=26, the loss=41738.311951274816
Current iteration=27, the loss=41738.18716743495
Current iteration=28, the loss=41738.17328558701
Current iteration=29, the loss=41738.171775992865
Current iteration=30, the loss=41738.171612228536
Current iteration=0, the loss=34776.580343053574
Current iteration=1, the loss=34762.09083532861
Current iteration=2, the loss=34747.63027726389
Current iteration=3, the loss=34733.19859459133
Current iteration=4, the loss=34718.79569253164
Current iteration=5, the loss=34690.06167273762
Current iteration=6, the loss=34647.15357526869
Current iteration=7, the loss=34590.311702481835
Current iteration=8, the loss=34505.80762891023
Current iteration=9, the loss=34380.73494656525
Current iteration=10, the loss=34203.601053760045
Current iteration=11, the loss=33951.96207164901
Current iteration=12, the loss=33596.107812161325
Current iteration=13, the loss=33117.034559561704
Current iteration=14, the loss=32478.120839237727
Current iteration=15,

Current iteration=12, the loss=48457.25141036411
Current iteration=13, the loss=43246.52849434658
Current iteration=14, the loss=39954.28570195004
Current iteration=15, the loss=39002.60127369778
Current iteration=16, the loss=38933.25605651912
Current iteration=17, the loss=38932.702655181994
Current iteration=0, the loss=53532.44990182514
Current iteration=1, the loss=53512.781232045076
Current iteration=2, the loss=53493.15169423484
Current iteration=3, the loss=53473.56127886527
Current iteration=4, the loss=53454.01036682841
Current iteration=5, the loss=53415.0060258811
Current iteration=6, the loss=53356.75968293351
Current iteration=7, the loss=53279.59928803463
Current iteration=8, the loss=53164.890147470665
Current iteration=9, the loss=52995.11305772573
Current iteration=10, the loss=52754.66915844956
Current iteration=11, the loss=52430.90777388029
Current iteration=12, the loss=51981.069431489086
Current iteration=13, the loss=51359.208539947045
Current iteration=14, the 

Current iteration=2, the loss=15289.527835819048
Current iteration=3, the loss=15283.351727713845
Current iteration=4, the loss=15271.03032330082
Current iteration=5, the loss=15252.631060700965
Current iteration=6, the loss=15222.178979831593
Current iteration=7, the loss=15174.010385565683
Current iteration=8, the loss=15097.16146210731
Current iteration=9, the loss=14976.650271020571
Current iteration=10, the loss=14790.61725700718
Current iteration=11, the loss=14511.80432141295
Current iteration=12, the loss=14120.890646970201
Current iteration=13, the loss=13613.421761768957
Current iteration=14, the loss=13029.48023819069
Current iteration=15, the loss=12458.356161782947
Current iteration=16, the loss=12010.935816752295
Current iteration=17, the loss=11756.978170472514
Current iteration=18, the loss=11671.642488004025
Current iteration=19, the loss=11660.020832590548
Current iteration=20, the loss=11659.64951065327
Current iteration=21, the loss=11659.64696975649
Current iterati

Current iteration=30, the loss=41769.4751980101
Current iteration=0, the loss=34768.95572406741
Current iteration=1, the loss=34754.4325338433
Current iteration=2, the loss=34739.9383578845
Current iteration=3, the loss=34725.473121914525
Current iteration=4, the loss=34711.03673125608
Current iteration=5, the loss=34682.23591469806
Current iteration=6, the loss=34639.228065789925
Current iteration=7, the loss=34582.25401185723
Current iteration=8, the loss=34497.55350590008
Current iteration=9, the loss=34372.19003998098
Current iteration=10, the loss=34194.644401547564
Current iteration=11, the loss=33942.42062585909
Current iteration=12, the loss=33585.7399247603
Current iteration=13, the loss=33105.55473763901
Current iteration=14, the loss=32465.161399244753
Current iteration=15, the loss=31660.001442330537
Current iteration=16, the loss=30698.046321118665
Current iteration=17, the loss=29626.086971557634
Current iteration=18, the loss=28536.25874247826
Current iteration=19, the l

KeyboardInterrupt: 

In [54]:
start = time.perf_counter()
preds = imp.train_predict_logistic(y_train, x_train, x_test)
end = time.perf_counter()

Current iteration=0, the loss=69254.41425128581
Current iteration=1, the loss=69208.90428501768
Current iteration=2, the loss=69163.4852396406
Current iteration=3, the loss=69072.873775137
Current iteration=4, the loss=68937.56558806098
Current iteration=5, the loss=68668.96439712941
Current iteration=6, the loss=68183.5734886181
Current iteration=7, the loss=67323.85835125927
Current iteration=8, the loss=65847.37097946732
Current iteration=9, the loss=63394.21364480965
Current iteration=10, the loss=59637.81539444657
Current iteration=11, the loss=54506.90721253042
Current iteration=12, the loss=48659.72626655982
Current iteration=13, the loss=43429.53963627368
Current iteration=14, the loss=40125.343566794705
Current iteration=15, the loss=39170.31738645596
Current iteration=16, the loss=39100.765210241785
Current iteration=17, the loss=39100.21091886141
Current iteration=0, the loss=53749.4049693404
Current iteration=1, the loss=53729.664765810565
Current iteration=2, the loss=5370

KeyboardInterrupt: 

In [45]:
helper.create_csv_submission(ids_test, preds, 'separated2.csv')

## Logistic regression

In [ ]:
niter = 5000

In [ ]:
start = time.perf_counter()
l, w, data = imp.stochastic_logistic_regression(y_train, std_x_train, max_iter=niter, batch_size=150, threshold=10**(-4))
end = time.perf_counter()

In [ ]:
print("Runtime:", end - start)

In [ ]:
len(data)

In [ ]:
plt.plot(data[100:])

In [ ]:
print("Loss first iteration", data[0])
print("Loss iteration at iteration", niter, data[-1])
print("delta of losses", data[0] - data[-1])

In [ ]:
std_x_test = np.hstack((np.ones((std_x_test.shape[0], 1)), std_x_test))                   

In [ ]:
predictions = helper.predict_labels(w, std_x_test)

In [ ]:
predictions

In [ ]:
helper.create_csv_submission(ids_test, predictions, 'newton.csv')

### Raw least squares (score: ?)

In [ ]:
(w, loss) = imp.least_squares(y_train, x_train)
loss

### Std least squares (score: 0.73)

In [ ]:
(w, loss) = imp.least_squares(y_train, col_std_x_train)
loss

In [ ]:
(w, loss) = imp.least_squares(y_train, std_x_train)
loss

### Eigenvalues reduction (score: 0.62)

In [ ]:
a = np.matrix([[1,2],[3,4]])
np.tile(a, 2)

In [ ]:
std_x_train = standardize(x_train)

In [ ]:
u, s, v = np.linalg.svd(std_x_train, full_matrices=False)
print('u shape:', u.shape)
print('s shape:', s.shape)
print('v shape:', v.shape)

In [ ]:
plt.plot(s)
plt.yscale('log')
plt.title('log')
plt.grid(True)
plt.show()

In [ ]:
shortened_x_train = u[:, :23] @ np.diag(s[:23]) @ v[:23,:]

In [ ]:
shortened_x_train.shape

In [ ]:
(w, loss) = imp.least_squares(y_train, x_train)

In [ ]:
loss

In [ ]:
y_pred = helper.predict_labels(w, x_test)

In [ ]:
print('Number of boson:', np.count_nonzero(y_pred+1))
print('Number of other:', np.count_nonzero(y_pred-1))

In [ ]:
helper.create_csv_submission(ids_test, y_pred, 'shortened_eigenvalues_submission.csv')

In [ ]:
unique_values_per_column_count = [len(set(col)) for col in train_data.T]

In [ ]:
unique_values_per_column_count

In [ ]:
def y_map(y):
    if y == -1:
        return 0
    else:
        return 0.2

y_train_mapped = np.vectorize(y_map)(y_train)

### PCA

In [ ]:
square_train_data = std_x_train.T @ std_x_train

In [ ]:
w, v = np.linalg.eigh(square_train_data)

In [ ]:
print(w.shape[0])
print(v.shape)

In [ ]:
w

In [ ]:
def keep_variance(percentage, vec):
    r = list(range(1, w.shape[0] + 1))
    total = np.sum(vec)
    sums = list(map(lambda i: np.sum(vec[-i:]), r))
    ratio = sums / total
    return np.argmin(abs(ratio - percentage)) + 1

In [ ]:
index_keeper = keep_variance(0.9, w)
print(index_keeper)

In [ ]:
plt.plot(w)
plt.yscale('log')
plt.title('log')
plt.grid(True)
plt.show()

In [ ]:
filtered_v = v[:,-index_keeper:]

In [ ]:
filtered_v.shape

In [ ]:
project_x_train = std_x_train @ filtered_v

In [ ]:
tupled_boson = np.array(list(zip(*filter(lambda pair: pair[1] == -1, zip(project_x_train.tolist(), y_train))))[0])
tupled_other = np.array(list(zip(*filter(lambda pair: pair[1] == 1, zip(project_x_train.tolist(), y_train))))[0])

In [ ]:
tupled_boson.shape

In [ ]:
plt.plot(tupled_boson[1000:2000, :1], tupled_boson[1000:2000, 1:], 'bo')
plt.plot(tupled_other[1000:2000, :1], tupled_other[1000:2000, 1:], 'ro')
plt.show()

In [ ]:
(weight, loss) = imp.least_squares(y_train, project_x_train)

In [ ]:
loss

In [ ]:
y_pred = helper.predict_labels(weight, std_x_test @ filtered_v)

In [ ]:
print('Number of boson:', np.count_nonzero(y_pred+1))
print('Number of other:', np.count_nonzero(y_pred-1))

### Polynomial feature

In [ ]:
np.array(range(2))

In [ ]:
def polynomial_enhancement(x, deg):
    stacked_x = np.tile(x, deg+1)
    power_vec = np.repeat(np.array(range(deg+1)), x.shape[1])
    return stacked_x ** power_vec

In [ ]:
enhanced_col_std_x_train = polynomial_enhancement(std_x_train, 9)
enhanced_col_std_x_test = polynomial_enhancement(std_x_test, 9)
enhanced_col_std_x_train.shape

In [ ]:
def pseudo_least_squares(y, x):
    U, S, V = np.linalg.svd(x, full_matrices=False)
    w = V.T @ np.diag(1/S) @ U.T @ y
    loss = imp.mse(y, x, w)
    return (w, loss)

In [ ]:
(w, loss) = pseudo_least_squares(y_train, enhanced_col_std_x_train)
loss

In [ ]:
y_pred = helper.predict_labels(w, enhanced_col_std_x_test)

In [ ]:
helper.create_csv_submission(ids_test, y_pred, 'basic_poly_enhancement_9.csv')

#### Cross validation

In [ ]:
acc, loss_train, loss_test, w = imp.cross_validation_v2(y_train, std_x_train, 10, imp.pseudo_least_squares, 5)

In [ ]:
acc

In [ ]:
enhanced_col_std_x_test = imp.polynomial_enhancement(std_x_test, 5)

In [ ]:
y_pred = helper.predict_labels(w, enhanced_col_std_x_test)

In [ ]:
helper.create_csv_submission(ids_test, y_pred, '0_8_accuracy_poly_5.csv')

In [ ]:
test = []

test.append(1)
test

In [ ]:
acc, loss_train, loss_test, w = imp.cross_validation_v2(y_train, std_x_train, 10, imp.pseudo_least_squares, 5)